# Unlocking Business Potential: Leveraging Bias-Aware Hierarchical Clustering for Actionable Insights from Yelp Data

### Zeinab Gaeini, Debbie Hernandez, Harper Strickland

## Part 2 - Prepare Data

### Consolidate and Format Business Data into Usable Dataframe of Normalized Features

#### This notebook has 2 outputs:
1. data csv file: all selected businesses, unpacked features in original format and scale
2. features csv file: selected businesses ids, stars, attributes normalized (0-1)

#### After running this notebook, get csv files from new local folders, rename, and move to local

---

#### Location to save results as csv

In [1]:
# UPDATE WITH NEW FILE NAMES - this should match final cell
save_features = 'hdfs:///Project/Features.csv'
save_data = 'hdfs:///Project/Data.csv'

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import col, split, when, coalesce   


In [3]:
# Suppress native-hadoop warning
!sed -i '$a\# Add the line for suppressing the NativeCodeLoader warning \nlog4j.logger.org.apache.hadoop.util.NativeCodeLoader=ERROR,console' /$HADOOP_HOME/etc/hadoop/log4j.properties

#### Terminal commands, Upload Files to HDFS

In [3]:
# make a directory on hdfs (repeated in case this notebook is used alone)
! hdfs dfs -mkdir /Project/

In [4]:
# copy files to hdfs
! hdfs dfs -copyFromLocal yelp_academic_dataset_business.json /Project
! hdfs dfs -copyFromLocal USZIPCodes202403.csv /Project
! hdfs dfs -copyFromLocal CanadianPostalCodes202403.csv /Project

In [ ]:
! hdfs dfs -ls /Project

#### Start Spark Session

In [5]:
# Start Spark Session
conf = pyspark.SparkConf().setAll([
        ('spark.master',   'local[*]'),
        ('spark.app.name', 'Prepare')])

spark = SparkSession.builder.config(conf=conf).getOrCreate()

spark.version


'3.5.0'

#### 2.1. Copy Yelp Business data file to PySpark dataframe

In [6]:
path = "hdfs:///Project/yelp_academic_dataset_business.json"
businessDF = spark.read.json(path, primitivesAsString=False)


### 2.2. Select businesses to include 
#### filteredDF: only open businesses of selected category

##### 2.2.1. Select subset of businesses

#### *UPDATE HERE:*


In [7]:
# SELECT DESIRED SUBSET OF BUSINESSES

# all open businesses (119698):
#filteredDF = businessDF.where(businessDF.is_open == 1)

# open Restaurants (34987):
filteredDF = businessDF.filter(businessDF.categories.contains('Restaurants')).where(businessDF.is_open == 1)

# open Gastropubs (331):
#filteredDF = businessDF.filter(businessDF.categories.contains('Gastropubs')).where(businessDF.is_open == 1)


In [9]:
print('All businesses:', businessDF.count(),
      '\nFiltered:', filteredDF.count())


All businesses: 150346 
Filtered: 34987


#### 2.2.2. Get latitude and longitude from reference dataset

(original dataset includes lat and long data for each state, not business locations)

In [10]:
# drop original latitude and longitude columns and other columns not needed
filteredDF = filteredDF.drop('latitude', 'longitude', 'address', 'is_open', 'hours').cache()


In [11]:
path = "hdfs:///Project/USZIPCodes202403.csv"
usa_ll = spark.read.csv(path, header=True) 


In [12]:
usa_ll = usa_ll.withColumn('ZipLatitude', usa_ll.ZipLatitude.cast(DoubleType()))
usa_ll = usa_ll.withColumn('ZipLongitude', usa_ll.ZipLongitude.cast(DoubleType()))


In [13]:
# Delete duplicate Zip Code rows
usa_ll = usa_ll.dropDuplicates(['Zip Code'])


In [14]:
# get USA latitude and longitude
filteredDF = filteredDF.join(usa_ll.select('Zip Code', 'ZipLatitude', 'ZipLongitude'),
                filteredDF.postal_code == usa_ll['Zip Code'], 'left')


In [15]:
path = "hdfs:///Project/CanadianPostalCodes202403.csv"
can_ll = spark.read.csv(path, header=True) 


In [16]:
can_ll = can_ll.withColumn('LATITUDE', can_ll.LATITUDE.cast(DoubleType()))
can_ll = can_ll.withColumn('LONGITUDE', can_ll.LONGITUDE.cast(DoubleType()))
can_ll = can_ll.withColumnRenamed('POSTAL_CODE', 'CA_POSTAL_CODE')


In [17]:
# get Canadian latitude and longitude
filteredDF = filteredDF.join(can_ll.select('CA_POSTAL_CODE', 'LATITUDE', 'LONGITUDE'),
                filteredDF.postal_code == can_ll.CA_POSTAL_CODE, 'left')


In [18]:
filteredDF = filteredDF.withColumn("lat",coalesce(filteredDF.ZipLatitude,filteredDF.LATITUDE)) 
filteredDF = filteredDF.withColumn("lon",coalesce(filteredDF.ZipLongitude,filteredDF.LONGITUDE)) 
# remove columns no longer needed
filteredDF = filteredDF.drop('Zip Code', 'ZipLatitude', 'ZipLongitude', 'CA_POSTAL_CODE',
                             'LATITUDE', 'LONGITUDE')
# delete rows with no latitude/longitude available
before = filteredDF.count()
filteredDF = filteredDF.dropna()
print('number of rows removed:', before - filteredDF.count())


number of rows removed: 512


In [19]:
# columns to be combined with normalized attributes
bus_cols = filteredDF.select('business_id', 'stars')


### 2.3. Unpack Attributes 
#### New DataFrame: attributesDF

**2.3.1.** Original format of attributes all in one column: StructField object, 39 elements, not all boolean


In [20]:
attributesDF = filteredDF.select(filteredDF.business_id, filteredDF.attributes)
for i in range (39):
    attributesDF = attributesDF.withColumn(filteredDF.schema['attributes'].dataType.names[i], 
                col('attributes.{}'.format(filteredDF.schema['attributes'].dataType.names[i])))
attributesDF = attributesDF.drop('attributes')

In [21]:
# drop columns with insignificant usage in any category:
#    AgesAllowed: 0.03% of all open businesses labeled over18 or over21
#    Open24Hours: 0.01% of all open businesses labeled true
#    RestaurantsCounterService: 0.01% of all open businesses labeled true
#    DietaryRestrictions: 0.01% of all open businesses used any available labels

attributesDF = attributesDF.drop('AgesAllowed', 'Open24Hours',
                                 'RestaurantsCounterService', 'DietaryRestrictions').cache()


**2.3.2.** Attributes to subdivide: Ambience, BestNights, BusinessParking, GoodForMeal, HairSpecializesIn, Music


In [22]:
# function to flatten attributes that have sub-attributes (stored as strings in varying orders)
def split_attribute(df, attr_name, attr_list):
    for i in range (len(attr_list)):
        # for each label, find string after label and before comma
        name = attr_name + '_' + attr_list[i]
        spl = "'" + attr_list[i] + "': "
        df = df.withColumn(name, split(split(split(attributesDF[attr_name], spl)[1], ",")[0], "}")[0])
    return df
    

In [23]:
ambience = ['divey', 'touristy', 'hipster', 'romantic', 'intimate', 'trendy', 'upscale', 'classy', 'casual']
attributesDF = split_attribute(attributesDF, 'Ambience', ambience)

In [24]:
bestnights = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
attributesDF = split_attribute(attributesDF, 'BestNights', bestnights)


In [25]:
businessparking = ['garage', 'street', 'validated', 'lot', 'valet']
attributesDF = split_attribute(attributesDF, 'BusinessParking', businessparking)


In [26]:
goodformeal = ['breakfast', 'brunch', 'lunch', 'dinner', 'latenight', 'dessert']
attributesDF = split_attribute(attributesDF, 'GoodForMeal', goodformeal)


In [27]:
hairspecializesin = ['straightperms', 'coloring', 'extensions', 'africanamerican', 'curly', 'kids', 'perms', 'asian']
attributesDF = split_attribute(attributesDF, 'HairSpecializesIn', hairspecializesin)


In [28]:
# not including 'no_music' option- zero businesses
# not including 'video' option- only 0.005% of open businesses
music = ['background_music', 'jukebox', 'dj', 'live', 'karaoke']
attributesDF = split_attribute(attributesDF, 'Music', music)


In [29]:
# remove columns no longer needed after flattening
attributesDF = attributesDF.drop('Ambience', 'BestNights', 'BusinessParking',
                                 'GoodForMeal', 'HairSpecializesIn', 'Music')


#### 2.3.3. New DF: Quantify and Normalize Attributes


In [30]:
# list of attributes with true,false values (to be normalized to 1,0)
true_cols = ['AcceptsInsurance', 'BYOB', 'BikeParking', 'BusinessAcceptsBitcoin',
             'BusinessAcceptsCreditCards', 'ByAppointmentOnly', 'Caters', 'CoatCheck', 'Corkage',
             'DogsAllowed', 'DriveThru', 'GoodForDancing', 'GoodForKids', 'HappyHour', 'HasTV',
             'OutdoorSeating', 'RestaurantsDelivery', 'RestaurantsGoodForGroups',
             'RestaurantsReservations', 'RestaurantsTableService', 'RestaurantsTakeOut',
             'WheelchairAccessible', 'Ambience_divey', 'Ambience_touristy', 'Ambience_hipster',
             'Ambience_romantic', 'Ambience_intimate', 'Ambience_trendy', 'Ambience_upscale',
             'Ambience_classy', 'Ambience_casual', 'BestNights_monday', 'BestNights_tuesday',
             'BestNights_wednesday', 'BestNights_thursday', 'BestNights_friday', 'BestNights_saturday',
             'BestNights_sunday', 'BusinessParking_garage', 'BusinessParking_street',
             'BusinessParking_validated', 'BusinessParking_lot', 'BusinessParking_valet',
             'GoodForMeal_breakfast', 'GoodForMeal_brunch', 'GoodForMeal_lunch', 'GoodForMeal_dinner',
             'GoodForMeal_latenight', 'GoodForMeal_dessert', 'HairSpecializesIn_straightperms',
             'HairSpecializesIn_coloring', 'HairSpecializesIn_extensions',
             'HairSpecializesIn_africanamerican', 'HairSpecializesIn_curly', 'HairSpecializesIn_kids',
             'HairSpecializesIn_perms', 'HairSpecializesIn_asian', 'Music_background_music',
             'Music_jukebox', 'Music_dj', 'Music_live', 'Music_karaoke'] 


In [31]:
# values for columns other than true/false
def normalize_column (df, col_name):
    name_list = ['Alcohol', 'BYOBCorkage', 'NoiseLevel', 'RestaurantsAttire',
                'RestaurantsPriceRange2', 'Smoking', 'WiFi']
    labels = [['beer_and_wine', 'full_bar'],
              ['yes_corkage', 'yes_free'],
              ['quiet', 'loud', 'very_loud'], # no need to check for average, same value as null
              ['dressy', 'formal'], # no need to check for casual, same value as null
              ['1', '2', '3', '4'],
              ['outdoor', 'yes'], # no need to check for no, same as null
              ['paid', 'free']] # no need to check for no, same as null
    # final in each list is value for otherwise:
    values = [[0.5, 1.0, 0.0],
              [0.5, 1.0, 0.0],
              [0.0, 0.67, 1.0, 0.33], # null noise level = average
              [0.5, 1.0, 0.0], # null attire = casual
              [0.0, 0.33, 0.67, 1.0, 0.5], # null price range = median of range
              [0.5, 1.0, 0.0], # null smoking = no
              [0.5, 1.0, 0.0]] # null wifi = no
    j = name_list.index(col_name)
    if j == 4: # RestaurantsPriceRange2: 4 labels
        df = df.withColumn(name_list[j], when(df[name_list[j]].contains(labels[j][0]), values[j][0]) \
                .when(df[name_list[j]].contains(labels[j][1]), values[j][1]) \
                .when(df[name_list[j]].contains(labels[j][2]), values[j][2]) \
                .when(df[name_list[j]].contains(labels[j][3]), values[j][3]).otherwise(values[j][4]))
    elif j == 2: # NoiseLevel: 3 labels
        df = df.withColumn(name_list[j], when(df[name_list[j]].contains(labels[j][0]), values[j][0]) \
                .when(df[name_list[j]].contains(labels[j][1]), values[j][1]) \
                .when(df[name_list[j]].contains(labels[j][2]), values[j][2]).otherwise(values[j][3]))
    else: # all others: 2 labels
        df = df.withColumn(name_list[j], when(df[name_list[j]].contains(labels[j][0]), values[j][0]) \
                .when(df[name_list[j]].contains(labels[j][1]), values[j][1]).otherwise(values[j][2]))
    return df


In [32]:
def normalize_attributes (df, true_list):
    for i in range (len(df.columns)):
        # business_id column untouched
        if (df.columns[i] == 'business_id'):
            pass
        # replace column values for true/false columns
        elif (df.columns[i] in true_list):
            df = df.withColumn(df.columns[i], when(df[df.columns[i]].like('True'), 1).otherwise(0))
        else:
            df = normalize_column(df, df.columns[i])
    return df

In [33]:
attr_norm = normalize_attributes(attributesDF, true_cols)


#### 2.4. Create Combined Dataframes: Business Data, and Normalized Features

**2.4.1.** Join business data dataframes and reorder columns

In [34]:
# join filteredDF and attributesDF
dataDF = filteredDF.join(attributesDF, filteredDF.business_id == attributesDF.business_id).drop(attributesDF.business_id)
dataDF = dataDF.drop('attributes')


In [35]:
# reorder columns
df_cols = ['business_id',
 'name',
 'stars',
 'review_count',
 'categories',
 'city',
 'state',
 'postal_code',
 'lat',
 'lon',
 'AcceptsInsurance',
 'Alcohol',
 'Ambience_casual',
 'Ambience_classy',
 'Ambience_divey',
 'Ambience_hipster',
 'Ambience_intimate',
 'Ambience_romantic',
 'Ambience_touristy',
 'Ambience_trendy',
 'Ambience_upscale',
 'BYOB',
 'BYOBCorkage',
 'BestNights_monday',
 'BestNights_tuesday',
 'BestNights_wednesday',
 'BestNights_thursday',
 'BestNights_friday',
 'BestNights_saturday',
 'BestNights_sunday',
 'BikeParking',
 'BusinessAcceptsBitcoin',
 'BusinessAcceptsCreditCards',
 'BusinessParking_garage',
 'BusinessParking_lot',
 'BusinessParking_street',
 'BusinessParking_valet',
 'BusinessParking_validated',
 'ByAppointmentOnly',
 'Caters',
 'CoatCheck',
 'Corkage',
 'DogsAllowed',
 'DriveThru',
 'GoodForDancing',
 'GoodForKids',
 'GoodForMeal_breakfast',
 'GoodForMeal_brunch',
 'GoodForMeal_lunch',
 'GoodForMeal_dinner',
 'GoodForMeal_dessert',
 'GoodForMeal_latenight',
 'HairSpecializesIn_africanamerican',
 'HairSpecializesIn_asian',
 'HairSpecializesIn_coloring',
 'HairSpecializesIn_curly',
 'HairSpecializesIn_extensions',
 'HairSpecializesIn_kids',
 'HairSpecializesIn_perms',
 'HairSpecializesIn_straightperms',
 'HappyHour',
 'HasTV',
 'Music_background_music',
 'Music_dj',
 'Music_jukebox',
 'Music_karaoke',
 'Music_live',
 'NoiseLevel',
 'OutdoorSeating',
 'RestaurantsAttire',
 'RestaurantsDelivery',
 'RestaurantsGoodForGroups',
 'RestaurantsPriceRange2',
 'RestaurantsReservations',
 'RestaurantsTableService',
 'RestaurantsTakeOut',
 'Smoking',
 'WheelchairAccessible',
 'WiFi']


In [36]:
dataDF = dataDF.select(df_cols)


**2.4.2.** Join normalized features dataframes and reorder columns

In [37]:
# join bus_cols and attr_norm
featuresDF = bus_cols.join(attr_norm, bus_cols.business_id == attr_norm.business_id).drop(attr_norm.business_id)


In [38]:
# remove columns from order that are not included in normalized attributes dataframe
remove = ['name', 'categories', 'city', 'state', 'postal_code', 'lat', 'lon', 'review_count']
df_cols = list(filter(lambda item: item not in remove, df_cols))


In [39]:
featuresDF = featuresDF.select(df_cols).cache()


**2.4.3.** Remove columns greater than 95% homogenous

In [40]:
# Delete columns over homogeneity threshold
# min different values required per column:
threshold = 0.05
num_rows = featuresDF.count()
# number of columns before removal:
before = len(featuresDF.columns)


In [41]:
# this step takes significant time to run (checking values of every row and column)
drop_cols = []

for i in range (1, len(featuresDF.columns)):

    count = featuresDF.groupBy(featuresDF[featuresDF.columns[i]]).count().orderBy('count', ascending=False).first()
    percent_same = count[1]/num_rows

    if (percent_same > 1 - threshold):
        drop_cols.append(featuresDF.columns[i])


In [42]:
featuresDF = featuresDF.drop(*drop_cols)
print('number of columns removed:', before - len(featuresDF.columns))
    

number of columns removed: 40


#### 2.5. Describe Features Dataframe

**2.5.1.** Display for all normalized attributes: possible values with counts of each

In [43]:
# Show counts of all values for features columns
for i in range (1, len(featuresDF.columns)):
    featuresDF.groupBy(featuresDF.columns[i]).count().orderBy('count', ascending=False).show(10)
    

+-----+-----+
|stars|count|
+-----+-----+
|  4.0| 9077|
|  3.5| 7248|
|  4.5| 5953|
|  3.0| 4526|
|  2.5| 3109|
|  2.0| 2126|
|  1.5| 1177|
|  5.0| 1075|
|  1.0|  184|
+-----+-----+

+-------+-----+
|Alcohol|count|
+-------+-----+
|    0.0|22482|
|    1.0| 8398|
|    0.5| 3595|
+-------+-----+

+---------------+-----+
|Ambience_casual|count|
+---------------+-----+
|              0|20804|
|              1|13671|
+---------------+-----+

+---------------+-----+
|Ambience_classy|count|
+---------------+-----+
|              0|29815|
|              1| 4660|
+---------------+-----+

+-----------------+-----+
|BestNights_friday|count|
+-----------------+-----+
|                0|32627|
|                1| 1848|
+-----------------+-----+

+-------------------+-----+
|BestNights_saturday|count|
+-------------------+-----+
|                  0|32654|
|                  1| 1821|
+-------------------+-----+

+-----------+-----+
|BikeParking|count|
+-----------+-----+
|          1|18357|
|       

**2.5.2.** Both Dataframes: size and schema 

In [44]:
# result dataframe size
(featuresDF.count(), len(featuresDF.columns))
# columns: id, stars = target/label, plus: of 72 features, all over threshold

(34475, 31)

In [45]:
featuresDF.printSchema()


root
 |-- business_id: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- Alcohol: double (nullable = false)
 |-- Ambience_casual: integer (nullable = false)
 |-- Ambience_classy: integer (nullable = false)
 |-- BestNights_friday: integer (nullable = false)
 |-- BestNights_saturday: integer (nullable = false)
 |-- BikeParking: integer (nullable = false)
 |-- BusinessAcceptsCreditCards: integer (nullable = false)
 |-- BusinessParking_lot: integer (nullable = false)
 |-- BusinessParking_street: integer (nullable = false)
 |-- Caters: integer (nullable = false)
 |-- DogsAllowed: integer (nullable = false)
 |-- DriveThru: integer (nullable = false)
 |-- GoodForKids: integer (nullable = false)
 |-- GoodForMeal_breakfast: integer (nullable = false)
 |-- GoodForMeal_brunch: integer (nullable = false)
 |-- GoodForMeal_lunch: integer (nullable = false)
 |-- GoodForMeal_dinner: integer (nullable = false)
 |-- HappyHour: integer (nullable = false)
 |-- HasTV: integer (nullable = f

In [46]:
# result dataframe size
(dataDF.count(), len(dataDF.columns))


(34475, 79)

In [47]:
dataDF.printSchema()


root
 |-- business_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- review_count: long (nullable = true)
 |-- categories: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- AcceptsInsurance: string (nullable = true)
 |-- Alcohol: string (nullable = true)
 |-- Ambience_casual: string (nullable = true)
 |-- Ambience_classy: string (nullable = true)
 |-- Ambience_divey: string (nullable = true)
 |-- Ambience_hipster: string (nullable = true)
 |-- Ambience_intimate: string (nullable = true)
 |-- Ambience_romantic: string (nullable = true)
 |-- Ambience_touristy: string (nullable = true)
 |-- Ambience_trendy: string (nullable = true)
 |-- Ambience_upscale: string (nullable = true)
 |-- BYOB: string (nullable = true)
 |-- BYOBCorkage: string (nullable = true)
 |-- BestNigh

#### 2.6. Save final dataframes as CSV files

In [48]:
featuresDF.coalesce(1).write.csv(save_features, mode='overwrite', header='true')
dataDF.coalesce(1).write.csv(save_data, mode='overwrite', header='true')



#### End Spark Session

In [49]:
# End Spark Session
spark.stop()


Copy csv file from HDFS to local

#### *UPDATE HERE:*

##### (both lines filename in Project folder should match name of .csv folders where file was saved in HDFS at top of notebook)


In [50]:
# UPDATE with folder name in HDFS (see output above)
! hdfs dfs -copyToLocal /Project/Features.csv Features_CSV_Folder
! hdfs dfs -copyToLocal /Project/Data.csv Data_CSV_Folder


##### After notebook runs, find the new file named “part-00000-…” in folder named “Features_CSV_Folder” Rename the “part-…” file as “Features.csv” and move it into local folder.

##### Similarly, find the new file named “part-00000-…” in folder named “Data_CSV_Folder” Rename the “part-…” file as “Data.csv” and move it into local folder.

##### Proceed to next notebook (3-Clustering.ipynb)

